<h1> Financial Risk - Machine Learning</h1>

O objetivo inicialdo estudo deste dataset é explorar o conjunto de dados  e desenvolver modelos preditivos que avaliem o risco de fornecer um seguro veicular. Para tanto, assumi que a seguradora deseja aumentar seus negócios em termos de escala, lucratividade e deseja alguns insights explorando os dados históricos. Primeiro e feito a previsao de ML da variavel TARGET_FLAG e logo para TARGET_AMT.

In [2]:
#Importando bibliotecas

#Manipulacao de dados
import numpy as np
import pandas as pd

#Visualizacao de dados
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import missingno
import warnings
warnings.filterwarnings('ignore')

#Sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
insurance_df = pd.read_csv('insurance_data.csv')

Machine Learning Pre Processamento

In [4]:
insurance_df.isnull().sum()

INDEX            0
TARGET_FLAG      0
TARGET_AMT       0
KIDSDRIV         0
AGE              6
HOMEKIDS         0
YOJ            454
INCOME         445
PARENT1          0
HOME_VAL       464
MSTATUS          0
SEX              0
EDUCATION        0
JOB            526
TRAVTIME         0
CAR_USE          0
BLUEBOOK         0
TIF              0
CAR_TYPE         0
RED_CAR          0
OLDCLAIM         0
CLM_FREQ         0
REVOKED          0
MVR_PTS          0
CAR_AGE        510
URBANICITY       0
dtype: int64

Removendo coluna index

In [5]:
insurance_df = insurance_df.drop(['INDEX'], axis=1)

Tratando valores nulos

In [6]:
insurance_df[insurance_df.AGE.isnull()].AGE 

239    NaN
1042   NaN
1314   NaN
2970   NaN
3459   NaN
4155   NaN
Name: AGE, dtype: float64

In [7]:
insurance_df['AGE'].fillna(insurance_df.AGE.mode()[0], inplace=True)

In [8]:
insurance_df[insurance_df.AGE.isnull()].AGE

Series([], Name: AGE, dtype: float64)

Idade do veiculo com valor negativo

In [9]:
insurance_df.loc[insurance_df['CAR_AGE'] == -3, ['CAR_AGE']] = insurance_df['CAR_AGE'].mode()

In [10]:
columns = ['INCOME','HOME_VAL','BLUEBOOK','OLDCLAIM']
insurance_df[insurance_df['INCOME'].isnull()]['INCOME']

3       NaN
28      NaN
53      NaN
60      NaN
63      NaN
       ... 
8084    NaN
8101    NaN
8104    NaN
8107    NaN
8136    NaN
Name: INCOME, Length: 445, dtype: object

In [11]:
for i in columns:
    insurance_df.drop(insurance_df[insurance_df[i].isnull()].index, axis=0, inplace=True)

In [12]:
insurance_df[insurance_df['INCOME'].isnull()]['INCOME']

Series([], Name: INCOME, dtype: object)

Removendo valores contábeis em campos numérico sendo necessario remover o sinal $ e as virgulas.

In [13]:
for i in columns:
  insurance_df[i] = insurance_df[i].str.replace("$", "")
  insurance_df[i] = insurance_df[i].str.replace(",", "").astype(int)

Removendo valores nulos

In [14]:
columns2 = ['YOJ','JOB','CAR_AGE']
for i in columns2:
    insurance_df.drop(insurance_df[insurance_df[i].isnull()].index, axis=0, inplace=True)

Tratando dados que estao divergentes com disturbios como z_F e z_No

In [15]:
insurance_df.loc[insurance_df['MSTATUS'] == 'z_No', ['MSTATUS']] = 'No'
insurance_df.loc[insurance_df['SEX'] == 'z_F', ['SEX']] = 'F'
insurance_df.loc[insurance_df['EDUCATION'] == 'z_High School', ['EDUCATION']] = 'High School'
insurance_df.loc[insurance_df['EDUCATION'] == '<High School', ['EDUCATION']] = 'High School'
insurance_df.loc[insurance_df['JOB'] == 'z_Blue Collar', ['JOB']] = 'Blue Collar'
insurance_df.loc[insurance_df['CAR_TYPE'] == 'z_SUV', ['CAR_TYPE']]  = 'SUV'
insurance_df.loc[insurance_df['URBANICITY'] == 'z_Highly Rural/ Rural', ['URBANICITY']]  = 'Highly Rural/ Rural'

Mudando variaveis as variaveis categoricas para numericas.

In [24]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6047 entries, 0 to 8160
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   TARGET_FLAG  6047 non-null   int64  
 1   TARGET_AMT   6047 non-null   float64
 2   KIDSDRIV     6047 non-null   int64  
 3   AGE          6047 non-null   float64
 4   HOMEKIDS     6047 non-null   int64  
 5   YOJ          6047 non-null   float64
 6   INCOME       6047 non-null   int64  
 7   PARENT1      6047 non-null   object 
 8   HOME_VAL     6047 non-null   int64  
 9   MSTATUS      6047 non-null   object 
 10  SEX          6047 non-null   object 
 11  EDUCATION    6047 non-null   object 
 12  JOB          6047 non-null   object 
 13  TRAVTIME     6047 non-null   int64  
 14  CAR_USE      6047 non-null   object 
 15  BLUEBOOK     6047 non-null   int64  
 16  TIF          6047 non-null   int64  
 17  CAR_TYPE     6047 non-null   object 
 18  RED_CAR      6047 non-null   object 
 19  OLDCLA

In [25]:
labelencoder1 = LabelEncoder()
insurance_df['PARENT1'] = labelencoder1.fit_transform(insurance_df['PARENT1'])

labelencoder2 = LabelEncoder()
insurance_df['MSTATUS'] = labelencoder2.fit_transform(insurance_df['MSTATUS'])

labelencoder3 = LabelEncoder()
insurance_df['SEX'] = labelencoder3.fit_transform(insurance_df['SEX'])

labelencoder4 = LabelEncoder()
insurance_df['EDUCATION'] = labelencoder4.fit_transform(insurance_df['EDUCATION'])

labelencoder5 = LabelEncoder()
insurance_df['JOB'] = labelencoder5.fit_transform(insurance_df['JOB'])

labelencoder6 = LabelEncoder()
insurance_df['CAR_USE'] = labelencoder6.fit_transform(insurance_df['CAR_USE'])

labelencoder7 = LabelEncoder()
insurance_df['CAR_TYPE'] = labelencoder7.fit_transform(insurance_df['CAR_TYPE'])

labelencoder8 = LabelEncoder()
insurance_df['RED_CAR'] = labelencoder8.fit_transform(insurance_df['RED_CAR'] )

labelencoder9 = LabelEncoder()
insurance_df['REVOKED'] = labelencoder9.fit_transform(insurance_df['REVOKED'])

labelencoder10 = LabelEncoder()
insurance_df['URBANICITY'] = labelencoder10.fit_transform(insurance_df['URBANICITY'])

Modelo com dados de maior influencia econtrado na analise exploratoria para TARGET_FLAG.

In [28]:
columns = ['HOMEKIDS', 'OLDCLAIM', 'CLM_FREQ', 'MVR_PTS', 'PARENT1', 'MSTATUS', 'EDUCATION', 'JOB', 'CAR_USE', 'CAR_TYPE', 'REVOKED', 'URBANICITY']
insurance_ml_df = insurance_df[columns]

In [30]:
x = insurance_ml_df
y = insurance_df['TARGET_FLAG']

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)

Modelo com todos os dados para TARGET_FLAG.

In [69]:
x2 = insurance_df.drop(['TARGET_FLAG','TARGET_AMT'], axis=1)
y2 = insurance_df['TARGET_FLAG']

In [70]:
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size = 0.3)

<h1>Machine Learning - Target Flag 1 e 2<h1>

In [85]:
model = RandomForestClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = RandomForestClassifier() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.97466
Acurácia 2: 1.0
Resultado 1: 91.66667%
Resultado 2: 100.0%


In [84]:
model = LinearSVC() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = LinearSVC() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.73719
Acurácia 2: 0.73333
Resultado 1: 8.33333%
Resultado 2: 0.0%


In [83]:
model = LogisticRegression() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = LogisticRegression()
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.7449
Acurácia 2: 0.7416
Resultado 1: 58.33333%
Resultado 2: 0.0%


In [82]:
model = ExtraTreesClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = ExtraTreesClassifier()
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.97466
Acurácia 2: 1.0
Resultado 1: 83.33333%
Resultado 2: 100.0%


In [77]:
model = KNeighborsClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = KNeighborsClassifier()
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.80606
Acurácia 2: 0.79008
Resultado 158.33333%
Resultado 285.71429%


In [78]:
model = GaussianNB() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = GaussianNB() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.73994
Acurácia 2: 0.7405
Resultado 1: 83.33333%
Resultado 2: 71.42857%


In [79]:
model = Perceptron() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = Perceptron() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.73333
Acurácia 2: 0.71625
Resultado 1: 16.66667%
Resultado 2: 85.71429%


In [80]:
model = SGDClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = SGDClassifier() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.73609
Acurácia 2: 0.73994
Resultado 1: 25.0%
Resultado 2: 0.0%


In [81]:
model = DecisionTreeClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia 1: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])

model = DecisionTreeClassifier() 
model.fit(x2_test, y2_test)
result = model.score(x2_test, y2_test)
print(f"Acurácia 2: {round(result,5)}")

t11 = np.array(y2_test[205:255])
t22 = model.predict(x2_test[205:255])

print(f'Resultado 1: {round((t2.sum())/(t1.sum())*100,5)}%')
print(f'Resultado 2: {round((t22.sum())/(t11.sum())*100,5)}%')

Acurácia 1: 0.97466
Acurácia 2: 1.0
Resultado 1: 83.33333%
Resultado 2: 100.0%


<h1>Machine Learning - Target Flag 2<h1>

In [ ]:
model = RandomForestClassifier() 
model.fit(x_test, y_test)
result = model.score(x_test, y_test)
print(f"Acurácia: {round(result,5)}")

t1 = np.array(y_test[205:255])
t2 = model.predict(x_test[205:255])
print(f'Resultado {round((t2.sum())/(t1.sum())*100,5)}%')